In [1]:
from pathlib import Path
import datetime as dt
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np

OUT_DIR = Path('../output')

In [6]:
BLANK = '?'
VOLUNTEERS_BY_WEEKDAY = {
    'Thursday': ['Tommy', 'Etienne'],
    'Sunday': ['Alex M', BLANK],
    }
MEETING_TIME_U = ('Sunday', 1, '15:00', '16:00')
MEETING_TIME_W = ('Wednesday', 1, '19:30', '20:30')
MEETING_TIME_R = ('Thursday', 1, '19:30', '20:30')

def build_roster(year, month, volunteers_by_weekday=VOLUNTEERS_BY_WEEKDAY, 
  meeting_time=None, blank=BLANK):
    """
    Given a year (integer) and a month (integer), build the roster for
    that year and month as a DataFrame with the columns
    
    - date: date of a shift
    - weekday: weekday of the shift, e.g. 'Sunday'
    - volunteers: names of volunteers working the shift
    - notes: any notes about the shift
    
    Give a dictionary of volunteers by weekday to seed the roster.
    Optionally specify a time for the monthly meeting.
    """
    # Create roster dates
    d1 = dt.date(year, month, 1)
    d2 = d1 + relativedelta(months=1)
    ix = pd.date_range(d1, d2, closed='left')
    f = pd.DataFrame(index=ix)
    f['date'] = f.index.strftime('%Y-%m-%d')
    f['weekday'] = f.index.weekday_name
    
    # Assign volunteers
    f['volunteers'] = f['weekday'].map(lambda x:
      ' & '.join(volunteers_by_weekday.get(x, [BLANK, BLANK])))
    
    # Restrict to given weekdays
    cond = f['weekday'].isin(volunteers_by_weekday.keys())
    f = f[cond].copy()
    
    # Add notes, including staff meeting.
    f['notes'] = ''
    if meeting_time is not None:
        cond = f['weekday'] == meeting_time[0]
        cond &= f.index.day >= 7*(meeting_time[1] - 1)
        cond &= f.index.day <= 7*meeting_time[1]
        f.loc[cond, 'notes'] = 'Staff meeting {!s}--{!s}'.format(*meeting_time[2:])
    
    return f.reset_index(drop=True)

def get_year_month(roster, format="%Y%m"):
    """
    Given a roster, return the year and month it pertains to 
    in the given format.
    """
    return pd.to_datetime(roster['date']).dt.strftime(format).iat[0]

def to_markdown(df, path=None):
    """
    Given DataFrame, return its corresponding Markdown table (string).
    If a path is given (string or Path object), then instead save the
    Markdown table to a Markdown file located at the path.
    """
    if path is not None:
        path = Path(path)
        if not path.parent.exists():
            path.parent.mkdir(parents=True)
    
    # Get column names
    cols = df.columns

    # Create a new DataFrame with just the markdown
    # strings
    df2 = pd.DataFrame([['---',]*len(cols)], columns=cols)

    #Create a new concatenated DataFrame
    df3 = pd.concat([df2, df])

    #Save as markdown
    if path is not None:
        df3.to_csv(path, sep="|", index=False)
    else:
        return df3.to_csv(sep="|", index=False)


In [8]:
roster = build_roster(2018, 1, meeting_time=None)
print(roster)

date = get_year_month(roster)
path = OUT_DIR/'roster_{!s}.md'.format(date)
to_markdown(roster, path)


         date   weekday       volunteers notes
0  2018-01-04  Thursday  Tommy & Etienne      
1  2018-01-07    Sunday       Alex M & ?      
2  2018-01-11  Thursday  Tommy & Etienne      
3  2018-01-14    Sunday       Alex M & ?      
4  2018-01-18  Thursday  Tommy & Etienne      
5  2018-01-21    Sunday       Alex M & ?      
6  2018-01-25  Thursday  Tommy & Etienne      
7  2018-01-28    Sunday       Alex M & ?      


'date|weekday|volunteers|notes\n---|---|---|---\n2018-01-04|Thursday|Tommy & Etienne|\n2018-01-07|Sunday|Alex M & ?|\n2018-01-11|Thursday|Tommy & Etienne|\n2018-01-14|Sunday|Alex M & ?|\n2018-01-18|Thursday|Tommy & Etienne|\n2018-01-21|Sunday|Alex M & ?|\n2018-01-25|Thursday|Tommy & Etienne|\n2018-01-28|Sunday|Alex M & ?|\n'